In [233]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from gensim.models import KeyedVectors

import tensorflow as tf
import torch
from torch import nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import string

In [234]:
# Setup some variables

tknzr = TweetTokenizer()
stemmer = PorterStemmer()
nltk.download('punkt')

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


----------------------------------------------------------------------

## Declaring some functions

In [235]:
# função de limpar texto
def cleanText(words, stem=False):
  """
    Esta função recebe um text e retorna o mesmo, já tratado com stopwords & punctuation
  """
  newWords = list()
  pontuacao = string,
  for word in words:
    word = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', word)
    words = re.sub("(@[A-Za-z0-9_]+)","", word)
    if len(word) > 0 and words not in string.punctuation and word.lower() not in stopwords and word.lower != "<br />":
      if stem:
        word = stemmer.stem(word.lower())
        newWords.append(word)
      else:
        newWords.append(word.lower())

  return newWords

In [236]:
# confusion matrix daora
def plot_conf_mat(y_test, y_preds, norm="false"):
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds, normalize=norm),
                    annot=True,
                    cbar=False)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")

In [237]:
def plot_loss_and_accuracy(losses, accs):

  fig, ax_tuple = plt.subplots(1, 2, figsize=(16,6))
  fig.suptitle('Loss and accuracy')

  for i, (y_label, y_values) in enumerate(zip(['BCE loss','Accuracy'],[losses, accs])):
    ax_tuple[i].plot(range(len(y_values)),  y_values, label='train')
    ax_tuple[i].set_xlabel('epochs')
    ax_tuple[i].set_ylabel(y_label)
    ax_tuple[i].legend()

In [238]:
def label2Embedding(sentence):
  for word in sentence: 
    if word in modelo.vocab:
      embed = modelo.get_vector(word)
      print(embed)
      return embed
    else:
      return 0
      #print("This word is not in the vocabuylary: ", word, "\n")

-------------------------------------------------------------------------------

### Importando o arquivo do GloVE de 50 dimensões e criando a variável *modelo* que será por onde iremos interagir com o *word embedding* já treinado

In [239]:
filename_txt = '../data/glove.6B.50d.txt'
modelo = KeyedVectors.load_word2vec_format(filename_txt)

### Lendo o dataset

In [240]:
df = pd.read_csv("../data/data.csv", encoding = "ISO-8859-1")
df.tail()

,Sl no,Tweets,Search key,Feeling
10012,10016,"Tweet #85: @Matteo tweeted ""@GameSpot @Frannkc...",irritating,angry
10013,10017,Tweet #86: @ðð§ð¢ð¬ð­ð¨ð§ tweet...,irritating,angry
10014,10018,"Tweet #87: @Chowkidar Ricky Sharma tweeted ""@M...",irritating,angry
10015,10019,"Tweet #88: @Katoe.EXE tweeted ""u know what i h...",irritating,angry
10016,10019,"Tweet #88: @Katoe.EXE tweeted ""u know what i h...",irritating,angry


### Copiando o dataset e fazendo transformações necessárias

In [241]:
dataset = df.copy()

dataset = dataset.drop(columns=["Sl no", "Search key"])
dataset.head()

,Tweets,Feeling
0,"#1: @fe ed ""RT @MirayaDizon1: Time is ticking...",happy
1,"#2: @è®è± &ã¯ãã ed ""RT @ninjaryugo: ï¼...",happy
2,"#3: @Ris â¡ ed ""Happy birthday to one smokin...",happy
3,#4: @ìì [ìì¯´ì¬ëë¡ë´] jwinnie is t...,happy
4,"#5: @Madhurima wth u vcâ¥ ed ""Good morning d...",happy


### Criando uma categoria com o *pd.Categorical* para cada emoção na tabela de emoções, dessa forma, teremos algo do tipo: happy - 1 | angry - 2 | sad = 3

Aplicando essas categorias na coluna *emotion_code*

In [242]:
dataset['Feeling'] = pd.Categorical(dataset['Feeling'])
dataset['emotion_code'] = dataset['Feeling'].cat.codes
dataset.head()

,Tweets,Feeling,emotion_code
0,"#1: @fe ed ""RT @MirayaDizon1: Time is ticking...",happy,3
1,"#2: @è®è± &ã¯ãã ed ""RT @ninjaryugo: ï¼...",happy,3
2,"#3: @Ris â¡ ed ""Happy birthday to one smokin...",happy,3
3,#4: @ìì [ìì¯´ì¬ëë¡ë´] jwinnie is t...,happy,3
4,"#5: @Madhurima wth u vcâ¥ ed ""Good morning d...",happy,3


### Tokenizando os tweets com o *tknzr.tokenize*, e logo após, limpando os tokens com a função *cleanText*

In [243]:
dataset["CleanText"] = [tknzr.tokenize(word) for word in dataset["Tweets"]]

dataset["CleanText"] = [cleanText(word) for word in dataset["CleanText"]]
dataset.head()

,Tweets,Feeling,emotion_code,CleanText
0,"#1: @fe ed ""RT @MirayaDizon1: Time is ticking...",happy,3,"[1, ed, rt, time, ticking, fast, relive, past,..."
1,"#2: @è®è± &ã¯ãã ed ""RT @ninjaryugo: ï¼...",happy,3,"[2, @è, , ®, è, , ±, ã, , ¯, ã, , , ã, ,..."
2,"#3: @Ris â¡ ed ""Happy birthday to one smokin...",happy,3,"[3, â, , ¡, ed, happy, birthday, one, smokin,..."
3,#4: @ìì [ìì¯´ì¬ëë¡ë´] jwinnie is t...,happy,3,"[4, @ì, , , ì, , , ì, , , ì, ¯, ´, ì, ,..."
4,"#5: @Madhurima wth u vcâ¥ ed ""Good morning d...",happy,3,"[5, wth, u, vcâ, , ¥, ed, good, morning, dear..."


### Criando o vocabulário...

In [244]:
#Criando EL VOCABULÁRIO (Com ajuda do código do sor em: Introdução ao PyTorch: da Regressão Linear à NLP com word-embeddings)
vocab_set = set() # será usado para gerar o vocabulário principal
max_len_doc = 0   # vamos medir o maior comprimento das mensagens envidas, em número de tokens
sum_len_doc = 0   # vamos medir o valor médio de palavras (tokens) por mensagem
min_word_len = 3  # comprimento mínimo de um token (em número de caracteres) para entrar no vocabulário 
tokens_list = []  # salvar a lista de tokens

for doc in dataset['CleanText']: # para cada documento do dataset
  #Pegando a palavra apenas se ela é maior que o comprimento mínimo
  for word in doc: 
    if len(word)>=min_word_len:
      tokens_list.append(word)

  # uso da função set: cria um conjunto dos elementos únicos da lista
  tokens_set = set(tokens_list) # i.e x = set(('text1', 'text2', 'text3'))  |  x => {'text1', 'text2', 'text3'}aaaaaaaaaaaaaaaaa
  vocab_set = set.union(vocab_set, tokens_set) # adiciona elementos únicos que ainda não pertencem ao conjunto do vocabulário

  sum_len_doc += len(word)
  if len(word) > max_len_doc:
    max_len_doc=len(word)

print(f'Tamanho total do vocabulário: V={len(vocab_set)}')
print(f'Número de palavras do texto mais longo: {max_len_doc}')
print(f'Média de palavras por texto: {sum_len_doc/len(df):3.4f}')

Tamanho total do vocabulário: V=26975
Número de palavras do texto mais longo: 53
Média de palavras por texto: 3.6412


### Something

In [245]:
word2idx = dict({})        # inicializa o dicionário
word2idx['<OOV>'] = 0      # índice da tag "out of vocabulary" é 0
word2idx['<PAD>'] = 1      # índice da tag "padding token" é 1

for i, v in enumerate(sorted(vocab_set),start=2): # enumera o vocabulário em ordem alfabética, a partir do índice 2 | i.e x = ('apple', 'banana', 'cherry') | x => [(0, 'apple'), (1, 'banana'), (2, 'cherry')]
  word2idx[v] = i

# testando a conversão "word to index" com o dicionário:
print(f'index for "<PAD>": {word2idx["<PAD>"]}')
print(f'index for "action": {word2idx["action"]}')

index for "<PAD>": 1
index for "action": 3517


### Transforma as palavras ordenadas do dicionário de *index* para uma lista

In [246]:
idx2word = list(word2idx.keys()) # apenas transforma as chaves (palavras ordenadas) do dicionário word2idx em uma lista

# testando a conversão "index to word":
print(f'word for index 0:    {idx2word[0]}')
print(f'word for index 100": {idx2word[666]}')

word for index 0:    <OOV>
word for index 100": #flamingos


### Conversão dos textos em uma sequência de índices (correspondente ao token do texto)

Limitar o tamanho máximo de um texto com ***max_len*** (truncar mensagem) e completar com <PAD> todos os textos que não cumprirem esse tamanho, para então termos variáveis do mesmo tamanho

In [247]:
max_len = 53         # comprimento máximo da mensagem (em número de palavras)
encoded_docs = []    # inicializa a lista de documentos codificados

for token in dataset['CleanText']: # para cada token
  encoded_d = [label2Embedding(t) for t in token]

  # adiciona o padding, se necessário
  encoded_d += [word2idx['<PAD>']]*max(0, max_len-len(encoded_d)) 
  # trunca o documento e salva na lista de documentos codificados
  encoded_docs.append(encoded_d[:max_len]) 

len(encoded_docs)  

 -2.7443e-03 -1.8298e-02
 -2.8096e-01  5.5318e-01  3.7706e-02  1.8555e-01 -1.5025e-01 -5.7512e-01
 -2.6671e-01  9.2121e-01]
[ 1.1891e-01  1.5255e-01 -8.2073e-02 -7.4144e-01  7.5917e-01 -4.8328e-01
 -3.1009e-01  5.1476e-01 -9.8708e-01  6.1757e-04 -1.5043e-01  8.3770e-01
 -1.0797e+00 -5.1460e-01  1.3188e+00  6.2007e-01  1.3779e-01  4.7108e-01
 -7.2874e-02 -7.2675e-01 -7.4116e-01  7.5263e-01  8.8180e-01  2.9561e-01
  1.3548e+00 -2.5701e+00 -1.3523e+00  4.5880e-01  1.0068e+00 -1.1856e+00
  3.4737e+00  7.7898e-01 -7.2929e-01  2.5102e-01 -2.6156e-01 -3.4684e-01
  5.5841e-01  7.5098e-01  4.9830e-01 -2.6823e-01 -2.7443e-03 -1.8298e-02
 -2.8096e-01  5.5318e-01  3.7706e-02  1.8555e-01 -1.5025e-01 -5.7512e-01
 -2.6671e-01  9.2121e-01]
[ 1.1891e-01  1.5255e-01 -8.2073e-02 -7.4144e-01  7.5917e-01 -4.8328e-01
 -3.1009e-01  5.1476e-01 -9.8708e-01  6.1757e-04 -1.5043e-01  8.3770e-01
 -1.0797e+00 -5.1460e-01  1.3188e+00  6.2007e-01  1.3779e-01  4.7108e-01
 -7.2874e-02 -7.2675e-01 -7.4116e-01  7.5263e-0

10017

### Agora os textos são os documentos codificados

In [248]:
dataset['X'] = encoded_docs
dataset['X'].tail()

10012    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, [0.118...
10013    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10014    [0, 0, 0, 0, 0, 0, [0.21705, 0.46515, -0.46757...
10015    [0, 0, 0, 0, 0, 0, 0, [0.11891, 0.15255, -0.08...
10016    [0, 0, 0, 0, 0, 0, 0, [0.11891, 0.15255, -0.08...
Name: X, dtype: object

### Verificando um exemplo de uma frase codificada

In [249]:
msg_codificada_ex = dataset['X'].iloc[7]
print(f' Mensagem codificada {msg_codificada_ex} \n Comprimento: {len(msg_codificada_ex)}')

 Mensagem codificada [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, array([ 1.1891e-01,  1.5255e-01, -8.2073e-02, -7.4144e-01,  7.5917e-01,
       -4.8328e-01, -3.1009e-01,  5.1476e-01, -9.8708e-01,  6.1757e-04,
       -1.5043e-01,  8.3770e-01, -1.0797e+00, -5.1460e-01,  1.3188e+00,
        6.2007e-01,  1.3779e-01,  4.7108e-01, -7.2874e-02, -7.2675e-01,
       -7.4116e-01,  7.5263e-01,  8.8180e-01,  2.9561e-01,  1.3548e+00,
       -2.5701e+00, -1.3523e+00,  4.5880e-01,  1.0068e+00, -1.1856e+00,
        3.4737e+00,  7.7898e-01, -7.2929e-01,  2.5102e-01, -2.6156e-01,
       -3.4684e-01,  5.5841e-01,  7.5098e-01,  4.9830e-01, -2.6823e-01,
       -2.7443e-03, -1.8298e-02, -2.8096e-01,  5.5318e-01,  3.7706e-02,
        1.8555e-01, -1.5025e-01, -5.7512e-01, -2.6671e-01,  9.2121e-01],
      dtype=float32), 0, 0, 0, 0, 0, 0, 0, 0, array([ 1.1891e-01,  1.5255e-01, -8.2073e-02, -7.4144e-01,  7.5917e-01,
       -4.8328e-01, -3.1009e-01,  5.1476e-01, -9.8708e-01,  6.1757e-04,
       

### Agora, usaremos apenas as mensagens codificadas (vetores de variáveis categóricas, coluna do DataFrame 'X') e as saídas rotuladas em  das emoções  (coluna 'y' do DataFrame). Também vamos converter os objetos para arrays do numpy.

In [250]:
X = np.vstack(dataset['X'].apply(lambda x: np.array(x)))
Y = np.array(dataset['emotion_code']).reshape(-1,1)
X.shape, X[0].shape, Y.shape, Y[0].shape

<ipython-input-250-bc7c0cfc6a55>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.vstack(dataset['X'].apply(lambda x: np.array(x)))


((10017, 53), (53,), (10017, 1), (1,))

### Separando com train_test_split

In [251]:
train_size = 0.8    # percentual de exemplos para o treino

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,                       # dataset para ser dividido, entrada X e saída Y
                                                    train_size=train_size,     # percentual resevado para o treinamento
                                                    stratify=Y,                # estratificação para manter a distribuição dos rótulos igual entre treino e teste
                                                    shuffle=True)              # embaralhar os exemplos aleatoriamente

X_train.shape, X_test.shape

((8013, 53), (2004, 53))

### Criando o modelo classificador

In [207]:

class Torch_Mean_Layer(nn.Module):
  '''Camada personalizada: calcula a média do tensor dentrada sobre a dimensão 1 (colunas).
     Retorna um vetor linha, onde cada elemento é a média dos elementos da coluna correspondente do tensor de entrada.
  '''
  def forward(self, x, dim=1):
    print("-----------------------",x)
    x = torch.mean(x, dim=dim, keepdims=True)
    return x

class mood_classifier(nn.Module):
  '''Modelo classificador de emoções
  '''

  # ----------------------------------------------#
  # Método construtor
  def __init__(self, vocab_size, dim_embed, n_units): 
    super().__init__()  

    embedding_seq = [] # 
    ann_seq       = [] # 
    soft_seq      = []

    #---------------------------------------------------------------#
    # Embedding step: sequência de operações para converter X --> h
    embedding_seq.append(Torch_Mean_Layer())
    #---------------------------------------------------------------#

    #--------------------------------------------------------------------------#
    # ANN: Rede Neural Artifical Tradicional, com regressão logística na saída
    ann_seq.append(nn.Linear(dim_embed, n_units))
    ann_seq.append(nn.ReLU(inplace=True))
    ann_seq.append(nn.Linear(n_units, 6))
    
    #--------------------------------------------------------------------------#
    # Softmax :)
    soft_seq.append(nn.LogSoftmax(dim=1))

    #--------------------------------------------------------------------------#

    #--------------------------------------------------------------------------#
    # "merge" de todas as camamadas em uma layer sequencial 
    # (uma sequência para cada etapa)
    self.embedding = nn.Sequential(*embedding_seq)     # etapa de embedding 
    self.ann       = nn.Sequential(*ann_seq)           # etapa ANN
    self.soft      = nn.Sequential(*soft_seq)
    #--------------------------------------------------------------------------#


  def forward(self, x): 
    '''Processamento realizado ao chamar y=modelo(x)
    '''
    x = self.embedding(x)  # aplica a etapa de embedding
    x = self.ann(x)        # passa o embedding médio pelas camadas da ANN
    x = x.view(-1,6)
    x = self.soft(x)
    return x  #Adcionar o softmax

### Função que irá treinar 

In [208]:
def train_loop(model, data, max_epochs = 1000, print_iters = 5):
  X_train, Y_train = data
  losses = []
  accs = []
  for i in range(max_epochs): # para cada época

      #-----------------------------------#
      # INÍCIO DO WORKFLOW DO TREINAMENTO #
      # 
      Y_pred = model.forward(X_train)         # apresente os dados de entrada para o modelo, e obtenha a previsão    
      loss = criterion(Y_pred.view(-1, 6), Y_train.view(-1))       # calcule a perda (o custo, o erro)
      optimizer.zero_grad()                   # inicialize os gradientes
      loss.backward()                         # backpropagation sobre a perda atual (cálculo dos novos gradientes) 
      optimizer.step()                        # atualização dos parâmetros da rede utilizando a regra do otimizador escolhido
      # FIM DO WORKFLOW DO TREINAMENTO    #
      #-----------------------------------#

      # ------ Bloco Opcional ------ #
      # Salvando métricas
      losses.append(loss)                     # salvando a perda atual
      acc = calc_accuracy(Y_pred, Y_train)     # calcula a taxa de acerto atual
      accs.append(acc)
      
      # Imprimindo resultados parciais
      if i % print_iters ==0: # a cada 10 iterações
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}') 
      #-----------------------------------#

  #----------------------------------------------------------------------------# 
  print('\n# Finished training!')
  print(f'# --> epoch: {i}  \n# --> initial loss: {losses[0]:10.8f} ,  \n# --> accuracy: {acc:2.8f} , \n# --> final loss: {losses[-1]:10.8f}')
  
  # retornando resultados
  return model, losses, accs

# Redefinindo cálculo da taxa de acerto 
def calc_accuracy(y_pred, y_true):
  ''' Helper function para calcular a taxa de acerto deste exemplo.
  '''
  y_pred = torch.argmax(y_pred, dim=1)
  y_pred = y_pred.float()
  y_true = torch.squeeze(y_true) # tentar rexplicar dps
  y_pred = torch.squeeze(y_pred)
  num_hits  = torch.sum(y_pred==y_true).numpy()
  num_total =  float(y_true.numel())
  acc=  num_hits/num_total
  return acc

### Treinando com o modelo

Convertendo os dados para tensores, instanciar o modelo, definir a função custo e o otimizador

In [189]:
data_train = (torch.LongTensor(X_train), torch.FloatTensor(Y_train))

Model = mood_classifier(vocab_size=len(word2idx), dim_embed=50, n_units=100)
print(Model)

criterion = nn.NLLLoss() # cross entropy loss
optimizer = torch.optim.Adam(Model.parameters(), lr = 0.01) 

Model, losses, accs = train_loop(Model, data_train, max_epochs=330, print_iters=1) # note que o modelo é sobrescrito pela saída treinada

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

### Mostrando a *Confusion Matrix*

In [ ]:
sns.set(font_scale=1.5)

matriz_de_confusao = confusion_matrix(Y_pred, Y_test)

#criando list com as emoções
emotion_class = ['Angry','Disgust','Fear','Happy','Sad','Surprise']

df_matriz_de_confusao = pd.DataFrame(matriz_de_confusao, emotion_class, emotion_class)

plot_conf_mat(Y_test, Y_pred)

### Gráfico de *loss* e *accuracy*

In [ ]:
plot_loss_and_accuracy(losses, accs)